In [452]:
"""TVAE module."""

import numpy as np
import pandas as pd
import torch
from torch.nn import Linear, Module, Parameter, ReLU, Sequential, GELU
from torch.nn.functional import cross_entropy
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from ctgan.data_transformer import DataTransformer
from ctgan.synthesizers.base import BaseSynthesizer, random_state


class Encoder(Module):
    """Encoder for the TVAE.

    Args:
        data_dim (int):
            Dimensions of the data.
        compress_dims (tuple or list of ints):
            Size of each hidden layer.
        embedding_dim (int):
            Size of the output vector.
    """

    def __init__(self, data_dim, compress_dims, embedding_dim):
        super(Encoder, self).__init__()
        dim = data_dim
        seq = []
        for item in list(compress_dims):
            seq += [Linear(dim, item), GELU()]
            dim = item

        self.seq = Sequential(*seq)
        self.fc1 = Linear(dim, embedding_dim)
        self.fc2 = Linear(dim, embedding_dim)

    def forward(self, input_):
        """Encode the passed `input_`."""
        feature = self.seq(input_)
        mu = self.fc1(feature)
        logvar = self.fc2(feature)
        std = torch.exp(0.5 * logvar)
        return mu, std, logvar


class Decoder(Module):
    """Decoder for the TVAE.

    Args:
        embedding_dim (int):
            Size of the input vector.
        decompress_dims (tuple or list of ints):
            Size of each hidden layer.
        data_dim (int):
            Dimensions of the data.
    """

    def __init__(self, embedding_dim, decompress_dims, data_dim):
        super(Decoder, self).__init__()
        dim = embedding_dim
        seq = []
        for item in list(decompress_dims):
            seq += [Linear(dim, item), GELU()]
            dim = item

        seq.append(Linear(dim, data_dim))
        self.seq = Sequential(*seq)
        self.sigma = Parameter(torch.ones(data_dim) * 0.1)

    def forward(self, input_):
        """Decode the passed `input_`."""
        return self.seq(input_), self.sigma


def _loss_function(recon_x, x, sigmas, mu, logvar, output_info, factor):
    st = 0
    loss = []
    for column_info in output_info:
        for span_info in column_info:
            if span_info.activation_fn != 'softmax':
                ed = st + span_info.dim
                std = sigmas[st]
                eq = x[:, st] - torch.tanh(recon_x[:, st])
                loss.append((eq**2 / 2 / (std**2)).sum())
                loss.append(torch.log(std) * x.size()[0])
                st = ed

            else:
                ed = st + span_info.dim
                loss.append(
                    cross_entropy(
                        recon_x[:, st:ed], torch.argmax(x[:, st:ed], dim=-1), reduction='sum'
                    )
                )
                st = ed

    assert st == recon_x.size()[1]
    KLD = -0.5 * torch.sum(1 + logvar - mu**2 - logvar.exp())
    return sum(loss) * factor / x.size()[0], KLD / x.size()[0]


class TVAE(BaseSynthesizer):
    """TVAE."""

    def __init__(
        self,
        embedding_dim=32,
        compress_dims=(32, 1024),
        decompress_dims=(1024, 32),
        l2scale=1e-5,
        batch_size=500,
        epochs=300,
        loss_factor=2,
        cuda=False,
        verbose=True,
    ):
        self.embedding_dim = embedding_dim
        self.compress_dims = compress_dims
        self.decompress_dims = decompress_dims

        self.l2scale = l2scale
        self.batch_size = batch_size
        self.loss_factor = loss_factor
        self.epochs = epochs
        self.loss_values = pd.DataFrame(columns=['Epoch', 'Batch', 'Loss'])
        self.verbose = verbose

        if not cuda or not torch.cuda.is_available():
            device = 'cpu'
        elif isinstance(cuda, str):
            device = cuda
        else:
            device = 'cuda'

        self._device = torch.device(device)

    @random_state
    def fit(self, train_data, discrete_columns=()):
        """Fit the TVAE Synthesizer models to the training data.

        Args:
            train_data (numpy.ndarray or pandas.DataFrame):
                Training Data. It must be a 2-dimensional numpy array or a pandas.DataFrame.
            discrete_columns (list-like):
                List of discrete columns to be used to generate the Conditional
                Vector. If ``train_data`` is a Numpy array, this list should
                contain the integer indices of the columns. Otherwise, if it is
                a ``pandas.DataFrame``, this list should contain the column names.
        """
        self.transformer = DataTransformer()
        self.transformer.fit(train_data, discrete_columns)
        train_data = self.transformer.transform(train_data)
        dataset = TensorDataset(torch.from_numpy(train_data.astype('float32')).to(self._device))
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, drop_last=False)

        data_dim = self.transformer.output_dimensions
        self.encoder = Encoder(data_dim, self.compress_dims, self.embedding_dim).to(self._device)
        self.decoder = Decoder(self.embedding_dim, self.decompress_dims, data_dim).to(self._device)
        optimizerAE = Adam(
            list(self.encoder.parameters()) + list(self.decoder.parameters()), weight_decay=self.l2scale
        )

        self.loss_values = pd.DataFrame(columns=['Epoch', 'Batch', 'Loss'])
        iterator = tqdm(range(self.epochs), disable=(not self.verbose))
        if self.verbose:
            iterator_description = 'Loss: {loss:.3f}'
            iterator.set_description(iterator_description.format(loss=0))

        for i in iterator:
            loss_values = []
            batch = []
            for id_, data in enumerate(loader):
                optimizerAE.zero_grad()
                real = data[0].to(self._device)
                mu, std, logvar = self.encoder(real)
                eps = torch.randn_like(std)
                emb = eps * std + mu
                rec, sigmas = self.decoder(emb)
                loss_1, loss_2 = _loss_function(
                    rec,
                    real,
                    sigmas,
                    mu,
                    logvar,
                    self.transformer.output_info_list,
                    self.loss_factor,
                )
                loss = loss_1 + loss_2
                loss.backward()
                optimizerAE.step()
                self.decoder.sigma.data.clamp_(0.01, 1.0)

                batch.append(id_)
                loss_values.append(loss.detach().cpu().item())

            epoch_loss_df = pd.DataFrame({
                'Epoch': [i] * len(batch),
                'Batch': batch,
                'Loss': loss_values,
            })
            if not self.loss_values.empty:
                self.loss_values = pd.concat([self.loss_values, epoch_loss_df]).reset_index(
                    drop=True
                )
            else:
                self.loss_values = epoch_loss_df

            if self.verbose:
                iterator.set_description(
                    iterator_description.format(loss=loss.detach().cpu().item())
                )

    @random_state
    def sample(self, samples, path="../data/external/bank.csv"):
        """Sample data similar to the training data.

        Args:
            samples (int):
                Number of rows to sample.

        Returns:
            numpy.ndarray or pandas.DataFrame
        """
        self.decoder.eval()

        steps = samples // self.batch_size + 1
        data = []
        for _ in range(steps):
            embedding_data = pd.read_csv(path, sep=",")
            fake, sigmas = self.decoder(torch.tensor(embedding_data.to_numpy(), dtype=torch.float32))
            fake = torch.tanh(fake)
            data.append(fake.detach().cpu().numpy())

        data = np.concatenate(data, axis=0)
        data = data[:samples]
        return self.transformer.inverse_transform(data, sigmas.detach().cpu().numpy())

    def generate_latents(self, data):
        """Generate latent vectors from the input data using the trained encoder.

        Args:
            data (numpy.ndarray or pandas.DataFrame):
                Input data to encode. Must match the format used for training.

        Returns:
            numpy.ndarray: Latent vectors generated by the encoder.
        """
        self.encoder.eval()
        data = self.transformer.transform(data)
        data_tensor = torch.from_numpy(data.astype('float32')).to(self._device)
        
        with torch.no_grad():  # Disable gradient calculations
            mu, std, _ = self.encoder(data_tensor)  # Get mean and standard deviation from encoder
            eps = torch.randn_like(std)  # Sample from a standard normal distribution
            emb = eps * std + mu  # Reparameterization trick to sample from the latent space

        return emb.cpu().numpy() 
    
    def set_device(self, device):
        """Set the `device` to be used ('GPU' or 'CPU)."""
        self._device = device
        self.decoder.to(self._device)

In [453]:
def categorical_column_indices(metadata_dict):
    categorical_indices = []
    columns = metadata_dict.get('columns', {})
    column_names = list(columns.keys())[:-1]  # Exclude the last key
    for index, column_name in enumerate(column_names):
        column_data = columns[column_name]
        if column_data.get('sdtype') == 'categorical':
            categorical_indices.append(index)
    return categorical_indices

In [438]:
def generate_and_save_latent(model, source="../data/interim/bank_no_label.csv", path="../data/processed/bank,.csv"):
    DATA_PATH = source
    df = pd.read_csv(DATA_PATH, sep=",")
    actual_data = df #.iloc[:, :-1]
    # outcomes = df.iloc[:, -1]

    latents = []
    metadata = SingleTableMetadata()
    meta = metadata.detect_from_csv(source)

    discrete_columns = categorical_column_indices(metadata.to_dict())
    print(discrete_columns)
    model.fit(actual_data, discrete_columns)
    latents = model.generate_latents(actual_data)
    # unbatched_latent = torch.cat(latents, dim=0)

    latents_df = pd.DataFrame(latents) #(unbatched_latent)
    # outcomes_df = pd.DataFrame(outcomes)
    # Save DataFrame to a CSV file
    # data_with_outcomes = pd.concat([latents_df, outcomes_df], axis=1)

    # data_with_outcomes.to_csv(path, index=False)
    latents_df.to_csv(path, index=False)

In [439]:
def split_columns(df: pd.DataFrame):
    """
    Split the input DataFrame into two parts:
    - all columns except the last one
    - the last column

    Args:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    pd.DataFrame, pd.Series: A DataFrame with all columns except the last one, and the last column as a Series.
    """
    # All columns except the last one
    all_except_last = df.iloc[:, :-1]

    # The last column
    last_column = df.iloc[:, -1]
    all_except_last.to_csv("../data/interim/bank_no_label.csv", index=False)
    return all_except_last, last_column

In [441]:
data = pd.read_csv("../data/raw/bank.csv", sep=",")
_, label = split_columns(data)
no_label_data = pd.read_csv("../data/interim/bank_no_label.csv", sep=",")
model = TVAE(embedding_dim=no_label_data.shape[1], compress_dims=(32,1024), decompress_dims=(1024,32))
generate_and_save_latent(model)

[5, 7, 9, 10, 11]


Loss: -42.321: 100%|██████████| 300/300 [12:15<00:00,  2.45s/it]


In [446]:
latent_check = pd.read_csv("../data/processed/bank,.csv")
combined_df = pd.concat([latent_check, label], axis=1)
combined_df.to_csv("../data/processed/bank.csv")

In [451]:
gen_data = np.load("../central_backbone/exp/bank_latent/X_num_unnorm.npy")
df = pd.DataFrame(gen_data)
df.to_csv("../data/external/bank.csv")

In [454]:
gen_df = model.sample(5000)
gen_df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online
0,524,25,1,121,94858,2,4.229875,1,0,0,0,0,1
1,683,44,18,23,94935,1,1.781621,1,1,0,0,0,1
2,4320,33,6,31,94817,4,0.324909,2,1,0,0,0,1
3,447,27,4,83,95161,1,2.166897,1,1,0,0,0,1
4,623,32,13,33,95147,4,0.224853,3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4729,29,3,69,94742,4,0.543778,1,0,0,0,0,1
4996,4492,31,4,46,94437,1,1.079280,1,-1,0,0,0,1
4997,4309,63,35,67,95060,1,1.053149,3,1,0,0,0,1
4998,3326,66,40,14,94563,2,2.374530,3,0,0,0,0,1


In [457]:
real_df = pd.read_csv("../data/raw/bank.csv", sep=",")
real_df = real_df.iloc[:, :-1]
real_df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1


## Resemblance

In [458]:
from scipy.stats import pearsonr, ks_2samp
from scipy.spatial.distance import jensenshannon
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier

In [459]:
def compute_categorical_similarity(col_real, col_synthetic):
    # Compute Theil's U for categorical features
    p_real = pd.Series(col_real).value_counts(normalize=True)
    p_synthetic = pd.Series(col_synthetic).value_counts(normalize=True)
    u = (p_real * np.log(p_real / p_synthetic)).sum()
    return 1 - u
def column_similarity(real_data, synthetic_data):
    similarities = []
    for col_real, col_synthetic in zip(real_data, synthetic_data):
        correlation, _ = pearsonr(col_real, col_synthetic)
        similarity = correlation
        similarities.append(similarity)
    return np.mean(similarities)
def correlation_similarity(real_data, synthetic_data):
    real_corr = np.corrcoef(real_data, rowvar=False)
    synthetic_corr = np.corrcoef(synthetic_data, rowvar=False)
    print(synthetic_corr)
    correlation, _ = pearsonr(real_corr.flatten(), synthetic_corr.flatten())
    return correlation
def jensen_shannon_similarity(real_data, synthetic_data):
    similarities = []
    for col_real, col_synthetic in zip(real_data.T, synthetic_data.T):
        # Compute probability distributions and Jensen-Shannon divergence
        p_real = np.histogram(col_real, bins=10, density=True)[0]
        p_synthetic = np.histogram(col_synthetic, bins=10, density=True)[0]
        similarity = 1 - jensenshannon(p_real, p_synthetic)
        similarities.append(similarity)
    return np.mean(similarities)
def kolmogorov_smirnov_similarity(real_data, synthetic_data):
    similarities = []
    for col_real, col_synthetic in zip(real_data.T, synthetic_data.T):
        # Compute cumulative distributions and Kolmogorov-Smirnov distance
        similarity, _ = ks_2samp(col_real, col_synthetic)
        similarity = 1 - similarity
        similarities.append(similarity)
    return np.mean(similarities)
def propensity_mean_absolute_similarity(real_data, synthetic_data):
    # Train XGBoost classifier to discriminate between real and synthetic samples
    X = np.vstack([real_data, synthetic_data])
    y = np.concatenate([np.ones(len(real_data)), np.zeros(len(synthetic_data))])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    classifier = XGBClassifier()
    classifier.fit(X_train, y_train)
    # Compute mean absolute error of classifier probabilities
    y_pred_proba = classifier.predict_proba(X_test)[:, 1]
    error = mean_absolute_error(y_test, y_pred_proba)
    return 1 - error

In [460]:
def resemblance_measure(real_data, synthetic_data):
    resemblance_score = (
        column_similarity(real_data, synthetic_data) +
        # correlation_similarity(real_data, synthetic_data) +
        jensen_shannon_similarity(real_data, synthetic_data) +
        kolmogorov_smirnov_similarity(real_data, synthetic_data) +
        propensity_mean_absolute_similarity(real_data, synthetic_data)
    ) / 4
    print("Resemblance Score:", resemblance_score)

In [461]:
resemblance_measure(real_df.to_numpy(), gen_df.to_numpy())

Resemblance Score: 0.8636330978384504


In [417]:
correlation_similarity(real_df.to_numpy(), gen_df.to_numpy())

[[ 1.00000000e+00 -8.47259459e-03 -8.32575989e-03 -1.76947432e-02
   1.34315402e-02 -1.67972432e-02 -2.46751718e-02  2.14632093e-02
  -1.39199177e-02 -2.48011655e-02 -1.69723327e-02 -6.90940312e-03
  -2.52841007e-03  1.70282321e-02]
 [-8.47259459e-03  1.00000000e+00  9.94214857e-01 -5.52686182e-02
  -2.92162874e-02 -4.64176636e-02 -5.20121791e-02  4.13343834e-02
  -1.25385869e-02 -7.72561717e-03 -4.36242223e-04  8.04255215e-03
   1.37024021e-02  7.68103676e-03]
 [-8.32575989e-03  9.94214857e-01  1.00000000e+00 -4.65741777e-02
  -2.86255480e-02 -5.25631471e-02 -5.00765106e-02  1.31518129e-02
  -1.05815524e-02 -7.41309808e-03 -1.23213441e-03  1.03533312e-02
   1.38978996e-02  8.96744734e-03]
 [-1.76947432e-02 -5.52686182e-02 -4.65741777e-02  1.00000000e+00
  -1.64098117e-02 -1.57500785e-01  6.45983670e-01 -1.87524257e-01
   2.06806228e-01  5.02462292e-01 -2.61649670e-03  1.69738080e-01
   1.42059193e-02 -2.38500775e-03]
 [ 1.34315402e-02 -2.92162874e-02 -2.86255480e-02 -1.64098117e-02
  

/Users/arnobchowdhury/anaconda3/envs/distributed-3-9/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/arnobchowdhury/anaconda3/envs/distributed-3-9/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


ValueError: array must not contain infs or NaNs

In [443]:
latent_check = pd.read_csv("../data/processed/bank,.csv")
latent_check.shape

(5000, 13)